In [1]:
# Creates a monthly nautical almanac similar to Maskelyne's ca. 1800
# Makes a pdf for the month with lunar distances to the 10 lunar stars, sun, and planets every 3 hrs
# Only gives data for those stars that are in a useful range
# Gives a bit of data for the sun and planets
# Gives right ascension and declination of the moon every 3 hrs to calculate lunars to other stars

# Go to last block (Main Program) to set month and year for calculation

from skyfield.api import Topos, load, Star
import math
import pandas as pd
import calendar
from day_month_year import * # routines for getting day, month names, etc.
from deg_min_sec import *    # routines for printing dms stuff
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak, Indenter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import styles

import time

In [2]:
def addPageNumber(canvas, doc):
    """
    Add the page number to the PDF doc
    """
    page_num = canvas.getPageNumber()
    text = "     %s" % page_num
    canvas.drawRightString(7.87*inch, 0.79*inch, text)

def EastWest(moonRA, starRA):
    """
    Which side of the moon is a star on given the right ascensions of both
    """
#return 1 for the star being East of the moon, 2 for West}
#    var m, s, west, east : integer;
    m = math.trunc(moonRA)
    s = math.trunc(starRA)
    adder = 360 - m
    s = s + adder
    if (s >= 360): s = s - 360
    if ((s > 0) and (s < 180)): return 2
    else: return 1

def plog(value):
    """
    returns the proportional log of a value (value < 10800)
    """
    # proportional logs
    # plog(x) = log(10800) - log(x)   (there are 10800 sec in 3 hrs)
    # plog(t-t1) = plog(d-D1) - plog(D2-D1) 
    # t is time of interest, t1 is earlier time in almanac (e.g. IIIh)
    # d is lunar distance at time t
    # D1 is lunar dist at time t1, D2 is lunar distance 3 hours later
    num = math.log10(10800) - math.log10(math.trunc(value*60))
    st = str(num)
    return(st[2:6])
           
def equation_of_time(t):
    """
    Returns the difference between sun time and clock time in minutes
    """
    time_of_day = 24 + t.utc[3]+t.utc[4]/60+t.utc[5]/3600 # get clock time to nearest second
    astrometric = earth.at(t).observe(sun)                # get geocentric position of sun 
    apparent = earth.at(t).observe(sun).apparent()        # correct for speed of light
    sun_ra, dec, distance = apparent.radec(epoch='date')  # get right ascension of sun
    sun_sha = 360 - 15 * sun_ra.hours                     # get SHA of sun
    gha_aries = 15 * t.gmst                               # get GHA aries
    sun_gha = (sun_sha + gha_aries)                       # GHA of sun = SHA_sun + GHA_aries
    time_of_sun = 24 + sun_gha / 15                       # convert to time
    if time_of_sun > 49:                                  # we're one day ahead 
        time_of_sun = time_of_sun - 24                    # subtract a day
    if (time_of_day - time_of_sun > 0.3):                 # if clock is more than 20 minutes ahead of sun then
        time_of_day = time_of_day - 12                    # subtract 12 hours from clock
    elif (time_of_sun - time_of_day > 0.3):               # if sun is more than 20 minutes ahead of clock then
        time_of_sun = time_of_sun - 12                    # subtract 12 hours from sun
    return((time_of_sun - time_of_day))*60                # return difference between sun and clock in minutes

In [3]:
def get_lunars(iyear, imonth): 
    """
    Calculates angular separation between the moon and sun, 10 lunar stars, and planets
    iyear, imonth are the year and month that is calculated
    Ca. 1800, the maritime day started at noon. Lunars are calculated every 3 hrs. for each day
    Only keeps the angles that are within a useful range for sextant measurement
    Ideal is to have 1 body West of the Moon and 1 East for each day of the month
    """
    # lunar_dist[day,j,body] where j is hour (noon,3,6,...,21)
    abody_ra = [0 for x in bodies]   # arrays for ra and dec of bodies
    abody_ra_d = [0 for x in bodies]
    abody_dec = [0 for x in bodies]
    #bodies = [sun,hamal,aldebaran,pollux,regulus,spica,antares,altair,fomalhaut,markab,venus,mars,jupiter,saturn]
    dmonth = imonth
    #we're going to use apparent time so on the 1st and last day of month we need to watch out for date change
    for iday in range(1, days_in_month(imonth)+1):
        print("Calculating",iday,'/',days_in_month(imonth),'\r', end="") # slow routine so give update each day
        dday = iday
        for j in range(0, 8): # calculate at 8 times during the day
            ihour = j * 3     # every 3 hrs
            dhour = ihour + 12 #just use proper hours starting at midnight
            t = ts.utc(iyear, dmonth, dday, dhour, 0, 0.0)
            # need Greenwich mean time (UTC) to calculate eq. of time
            eqt = -1 * equation_of_time(t) 
            #now we can get apparent time by adding the eq of time
            t = ts.utc(iyear, dmonth, dday, dhour, eqt,0 ) # 
            # Greenwich apparent time (GAT)
 
            astrometric = earth.at(t).observe(moon)          # Skyfield routines to get data for body
            apparent = earth.at(t).observe(moon).apparent()  # allow for light speed
            ra, dec, distance = apparent.radec(epoch='date') # get RA, dec, and dist from earth
            moon_ra = ra
            moon_dec = dec
            moon_horizontal_parallax = ((math.degrees(math.asin(6378.14/(distance.au*149597870)))*60))
            moon_semi_diameter = ((358473400/(distance.au * 149597870))/60);
            moon_sha = 360 - 15 * moon_ra.hours # Sidereal hour angle of moon
            gha_aries = 15 * t.gmst             # Greenwich hour angle of first point of Aries
            moon_gha = (moon_sha + gha_aries)   # GHA of moon (not used?)
            if (moon_gha > 360):
                moon_gha = moon_gha - 360
 
            astrometric = earth.at(t).observe(sun)
            apparent = earth.at(t).observe(sun).apparent()
            ra, dec, distance = apparent.radec(epoch='date')
            sun_ra = ra
            sun_dec = dec
            sun_semi_diameter = (959.63/distance.au)/60
            sun_sha = 360 - 15 * sun_ra.hours
            sun_gha = (sun_sha + gha_aries)
            if (sun_gha > 360):
                sun_gha = sun_gha - 360 
            for k in range(14):  # get lunars for the sun, 10 stars and 4 planets
                body = lunar_stars[k]
                if (body != sun):
                    astrometric = earth.at(t).observe(body)
                    apparent = earth.at(t).observe(body).apparent()
                    ra, dec, distance = apparent.radec(t)
                abody_ra[k] = ra.radians
                abody_ra_d[k] = ra._degrees
                abody_dec[k] = dec.radians
                lunar_distance = math.degrees(math.acos((math.sin(moon_dec.radians) * math.sin(abody_dec[k])) + 
                                                        (math.cos(moon_dec.radians) * math.cos(abody_dec[k])*math.cos(abody_ra[k] - moon_ra.radians))))
                moon_is_visible = (abs(moon_ra._degrees - sun_ra._degrees) > 30) # is the moon hidden in the sun?
                if (body == sun):
                    if ((lunar_distance > 37) and (lunar_distance < 121)): # useable angles for taking lunars
                        lunar_dist[k][j][iday] = lunar_distance # record the lunar and direction if useable
                        lunar_dir[k][j][iday] = EastWest(moon_ra._degrees,abody_ra_d[k])
                    else: lunar_dist[k][j][iday] = 0 # if not useable, record a zero
                elif (body in [venus,mars,jupiter,saturn]):
                    if ((lunar_distance > 19) and (lunar_distance < 120) and (moon_is_visible)):
                    #if ((lunar_distance > 19) and (lunar_distance < 120) and (moon_is_visible) and (night_time(dmonth,j))):
                        lunar_dist[k][j][iday] = lunar_distance
                        lunar_dir[k][j][iday] = EastWest(moon_ra._degrees,abody_ra_d[k])
                    else: lunar_dist[k][j][iday] = 0
                else: #fixed stars
                    if (body == hamal): limits = [310,350,50,100]    # limits of useability are taken from early almanacs
                    if (body == aldebaran): limits = [340,40,100,140]# numbers are moon's RA
                    if (body == pollux): limits = [30,70,140,180]    # when moon is either W or E of body within the
                    if (body == regulus): limits = [60,120,170,220]  # limits of the given numbers then
                    if (body == spica): limits = [110,175,215,270]   # the lunar is useable
                    if (body == antares): limits = [170,215,260,355]
                    if (body == altair): limits = [200,240,350,10]
                    if (body == fomalhaut): limits = [230,300,5,30]
                    if (body == markab): limits = [290,320,20,50]
                    done = False
                    if limits[0] < limits[1]:
                        if ((moon_ra._degrees > limits[0]) and (moon_ra._degrees < limits[1]) and (moon_is_visible)):
                            lunar_dist[k][j][iday] = lunar_distance
                            lunar_dir[k][j][iday] =  EastWest(moon_ra._degrees,abody_ra_d[k]) #1
                            done = True
                        else: lunar_dist[k][j][iday] = 0
                    elif (done != True): #jump from 360 to 0
                        if ((((moon_ra._degrees > limits[0]) and (moon_ra._degrees <= 360)) or 
                        ((moon_ra._degrees >= 0) and (moon_ra._degrees < limits[1]))) and (moon_is_visible)):
                            lunar_dist[k][j][iday] = lunar_distance
                            lunar_dir[k][j][iday] =  EastWest(moon_ra._degrees,abody_ra_d[k]) #1
                            done = True
                        else: lunar_dist[k][j][iday] = 0
                    if ((limits[2] < limits[3]) and (done != True)):
                        if ((moon_ra._degrees > limits[2]) and (moon_ra._degrees < limits[3]) and (moon_is_visible)):
                            lunar_dist[k][j][iday] = lunar_distance
                            lunar_dir[k][j][iday] =  EastWest(moon_ra._degrees,abody_ra_d[k]) #0
                        else: lunar_dist[k][j][iday] = 0
                    elif (done != True): #jump from 360 to 0
                        if ((((moon_ra._degrees > limits[2]) and (moon_ra._degrees <= 360)) 
                            or ((moon_ra._degrees >= 0) and (moon_ra._degrees < limits[3]))) and (moon_is_visible)):
                            lunar_dist[k][j][iday] = lunar_distance
                            lunar_dir[k][j][iday] =  EastWest(moon_ra._degrees,abody_ra_d[k]) #0
                        else: lunar_dist[k][j][iday] = 0
    print()


In [4]:
def find_line(starname,date):
    """
    Not sure
    """
    line = []
    for i in range(len(printout)):
        line = printout[i]
        #if (line != None): 
        if ((starname == line[0]) and (date == int(line[1]))):
            return i
            break
            
def print_lunar_distances(direction, stars_only):
    """
    Prints a table of lunar distances for each day of the month
    direction indicates whether East or West of the moon
    stars_only indicates whether table is for stars or planets
    """
    printline = []
    count = 0
    data = [["" for x in range(33)] for y in range(14)]
    block_start = [[0 for x in range(14)] for y in range(2)] #starting day of lunars for body
    if (stars_only):
        if (direction == 1):
            #print("DISTANCES of Moon's Center from Sun, and from Stars WEST of her.")
            lstr = "DISTANCES of Moon's Center from Sun, and from Stars WEST of her."
        else: 
            #print("DISTANCES of Moon's Center from Sun, and from Stars EAST of her.")
            lstr = "DISTANCES of Moon's Center from Sun, and from Stars EAST of her."
        #print("Stars\tDays\tNoon\tIIIh\tVIh\tIXh\tMidn/t\tXVh\tXVIIIh\tXXIh")
    else:
        if (direction == 1):
            #print("DISTANCES of Moon's Center from Planets WEST of her.")
            lstr = "DISTANCES of Moon's Center from Planets WEST of her."
        else: 
            #print("DISTANCES of Moon's Center from Planets EAST of her.")
            lstr = "DISTANCES of Moon's Center from Planets EAST of her."
        #print("Planets\tDays\tNoon\tIIIh\tVIh\tIXh\tMidn/t\tXVh\tXVIIIh\tXXIh")
    #print("Names\t\tD.M.S.\tD.M.S.\tD.M.S.\tD.M.S.\tD.M.S.\tD.M.S.\tD.M.S.\tD.M.S.")
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    header = Paragraph(ptext, ps)
    #header = Paragraph(ptext, styleSheet['Normal'])
    elements.append(header)
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[6.44*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    for k in range(14): #look at all bodies
        block = 0 #possible to have 2 blocks of dates where lunars are within range
        for iday in range(1, days_in_month(imonth)+1): #for each day of the month
            printline = []
            st = star_name[k]+"\t"+str(iday)+"\t"
            printline.append(star_name[k])
            printline.append(str(iday))
            data_is_present = False 
            first_block_done = False
            for j in range(0, 8): #8# every 3 hours from noon until 9:00AM
                if ((lunar_dir[k][j][iday] == direction) and (lunar_dist[k][j][iday] != 0)):
                    data_is_present = True #there is a lunar distance and it's on the right side of the moon
                    if (block_start[0][k] == 0): block_start[0][k] = iday #first day with data
                    if ((block_start[1][k] == 0) and first_block_done): block_start[1][k] = iday #first day with data
                    #break
                else:
                    if (block_start[0][k] != 0): first_block_done = True
            if (data_is_present): #if body in range for this day and this body then (all day)
                for j in range(0, 8): #8#for every 3 hours
                    if (lunar_dir[k][j][iday] == direction): #body is on the correct side of the moon
                        if (k < 10):
                            if (lunar_dist[k][j][iday] > 0): #body is within range
                                st = st + dms_string(lunar_dist[k][j][iday]) + "\t" #add the distance to the line
                                printline.append(dms_string2(lunar_dist[k][j][iday]))
                            else:
                                st = st + "- - - \t" #add leading and trailing zeros for this line ONLY WEST
                                printline.append("- - -")
                                if ((block == 0) and (block_start[block][k] != 0)): block = 1
                        else:
                            if (lunar_dist[k][j][iday] > 0): #body is within range
                                st = st + dms_string(lunar_dist[k][j][iday]) + "\t" #add the distance to the line
                                printline.append(dms_string2(lunar_dist[k][j][iday]))
                            else:
                                st = st + "- - - \t" #add leading and trailing zeros for this line ONLY WEST
                                printline.append("- - -")
                                if ((block == 0) and (block_start[block][k] != 0)): block = 1
                    else: 
                        st = st + "- - - \t" #not sure why this is needed but it is for trailing zeros ONLY EAST
                        printline.append("- - -")
                data_is_present = False
                data[k][iday] = st #add the line of text to the page
                # print(st)
                if (count < 100): printout[count] = printline
                count += 1
    for x in range(1, days_in_month(imonth)): #need to write out the lunar distances in order of date
        for k in range(10): #each body stays together as a block of contiguous dates
            if ((block_start[0][k] == x) and (stars_only)): #if the start date of the block is this day then
                i = x #set i to the date
                while True: #repeat until a date with no data is reached
                    if data[k][i] != '': #if there is data on this date
                        if (i == x): #first day of block
                            #print(data[k][i]) #print it out
                            line_num = find_line(star_name[k],i)
                            if line_num != None: printout2.append(printout[line_num])
                        else: #delete star name from line
                            #print(data[k][i][len(star_name[k]):]) #print it out                            
                            line_num = find_line(star_name[k],i)
                            if line_num != None: printout2.append(printout[line_num])
                        i += 1 #go on to the next day
                    else: #see if there is another block of dates later in the month for this body
                        if (block_start[1][k] != 0): #if second block exists
                            block_start[0][k] = block_start[1][k] #stick it in the first block and
                            block_start[1][k] = 0 #erase the second block
                        break #go on to the next body and go back to date x
        for k in range(10,14): #each body stays together as a block of contiguous dates
            if ((block_start[0][k] == x) and (stars_only == False)): #if the start date of the block is this day then
                i = x #set i to the date
                while True: #repeat until a date with no data is reached
                    if data[k][i] != '': #if there is data on this date
                        if (i == x): #first day of block
                            #print(data[k][i]) #print it out
                            line_num = find_line(star_name[k],i)
                            if line_num != None: printout2.append(printout[line_num])
                        else: #delete star name from line
                            #print(data[k][i][len(star_name[k]):]) #print it out                            
                            line_num = find_line(star_name[k],i)
                            if line_num != None: printout2.append(printout[line_num])
                        i += 1 #go on to the next day
                    else: #see if there is another block of dates later in the month for this body
                        if (block_start[1][k] != 0): #if second block exists
                            block_start[0][k] = block_start[1][k] #stick it in the first block and
                            block_start[1][k] = 0 #erase the second block
                        break #go on to the next body and go back to date x
    table_breaks = [] #we want to put a box around all the days for each star
    for i in range(2,len(printout2)):
        star = printout2[i][0]
        laststar = printout2[i-1][0]
        if (star != laststar): #store the line number of the list when a transition from star to star happens
            table_breaks.append(i)
    j = len(table_breaks)
    for i in range(j,15): #assume that 15 rows is the max in one month
        table_breaks.append(table_breaks[j-1]) #just copy the last item up to the end
    #delete all duplicate names of each star
    for i in range(1,15):
        star = printout2[table_breaks[i]][0]
        midway = int((table_breaks[i]-table_breaks[i-1])/2)+table_breaks[i-1]
        for j in range(table_breaks[i-1],table_breaks[i]):
            if (j != midway):
                printout2[j][0] = ""
    star = printout2[table_breaks[14]][0]
    midway = int((len(printout2)-table_breaks[14])/2)+table_breaks[14]
    for j in range(table_breaks[14],len(printout2)):
        if (j != midway):
            printout2[j][0] = ""
    ttab=Table(printout2,[0.75*inch,0.25*inch,0.68*inch,0.68*inch,0.68*inch,0.68*inch,0.68*inch,0.68*inch,0.68*inch,0.68*inch],
               len(printout2)*[0.15*inch]) #create a table for the list
    table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                  ('ALIGN',(1,0),(-1,-1),'CENTER'),
                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                  ('INNERGRID', (0,0), (-1,2), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[1]-1), (-1,table_breaks[1]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[2]-1), (-1,table_breaks[2]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[3]-1), (-1,table_breaks[3]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[4]-1), (-1,table_breaks[4]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[5]-1), (-1,table_breaks[5]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[6]-1), (-1,table_breaks[6]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[7]-1), (-1,table_breaks[7]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[8]-1), (-1,table_breaks[8]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[9]-1), (-1,table_breaks[9]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[10]-1), (-1,table_breaks[10]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[11]-1), (-1,table_breaks[11]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[12]-1), (-1,table_breaks[12]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[13]-1), (-1,table_breaks[13]), 0.25, colors.black),
                  ('INNERGRID', (0,table_breaks[14]-1), (-1,table_breaks[14]), 0.25, colors.black),
                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                  ('BOX', (1,0), (2,-1), 0.25, colors.black),
                  ('BOX', (2,0), (3,-1), 0.25, colors.black),
                  ('BOX', (3,0), (4,-1), 0.25, colors.black),
                  ('BOX', (4,0), (5,-1), 0.25, colors.black),
                  ('BOX', (5,0), (6,-1), 0.25, colors.black),
                  ('BOX', (6,0), (7,-1), 0.25, colors.black),
                  ('BOX', (7,0), (8,-1), 0.25, colors.black),]
    ttab.setStyle(TableStyle(table_args))
    elements.append(ttab)
    elements.append(PageBreak())


In [5]:
def print_solar_month(iyear,imonth):
    """
    Prints a table of the sun's longitude, right ascension, declination, eq of time, and semi diameter
    Each value given for noon and midnight of each day
    """
    lstr = "The Sun's"
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    elements.append(Indenter(left=0.65*inch))
    header = Paragraph(ptext, ps)
    elements.append(header)
    elements.append(Indenter(left=-0.65*inch))
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[5.05*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    tabdata= [['day','date','Longtde','RA','Dec','Eq of time','diff','SD'],
             ['','','D.M.S','H.M.S','D.M.S','','','']]
    ttab=Table(tabdata,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch],2*[0.2*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black)]))
    elements.append(ttab)
    printout2 = []
    #print(calendar.month_abbr[imonth]+" "+str(iyear)+"  "+"The Sun's")
    #print('day\t date\t Longtde\t RA\t Dec\t Eq of time\t diff\t SD')
    for iday in range(1, days_in_month(imonth)+1):
        dday = iday
        dhour = 12 #noon
        t = ts.utc(iyear, imonth, dday, dhour, 0, 0.0)
        # Greenwich mean time (UTC)
        eqt = -1 * equation_of_time(t) 
        t = ts.utc(iyear, imonth, dday, dhour, eqt,0 ) 
        # Greenwich apparent time (GAT)
        eqt = equation_of_time(t)
        if (iday > 1): 
            diff = (eqt - old_diff) * 60
            old_diff = eqt
        else: 
            diff = 0
            old_diff = eqt
        if (abs(int((eqt-int(eqt))*60)) < 10): s_adder = '0'
        else: s_adder = ''
        eqt_str = str(int(eqt))+'m'+s_adder+str(abs(int((eqt-int(eqt))*60)))+'s'
        diff_str = str(int(diff))+'.'+str(abs(int((diff-int(diff))*10)))+'s'
        gha_aries = 15 * t.gmst
        astrometric = earth.at(t).observe(sun)
        apparent = earth.at(t).observe(sun).apparent()
        ra, dec, distance = apparent.radec(epoch='date')
        sun_ra = ra
        sun_dec = dec
        sun_semi_diameter = (959.63/distance.au)/60
        sd_str = str(int(sun_semi_diameter))+"' "+str(int((sun_semi_diameter - math.trunc(sun_semi_diameter))*60))+'"'
        sun_sha = 360 - 15 * sun_ra.hours
        sun_gha = (sun_sha + gha_aries)
        if (sun_gha > 360):
            sun_gha = sun_gha - 360
        eclat, eclon, ecd = earth.at(t).observe(sun).ecliptic_latlon()
        dow = weekDay(iyear, imonth, iday)
        #print(dow,"\t",iday,"\t",dms_string(eclon.degrees),"\t",hms_string(sun_ra.hours),"\t",
              #dms_string(sun_dec.degrees),"\t",eqt_str,"\t",diff_str,"\t",sd_str)
        print_line = [str(dow),str(iday),dms_string2(eclon.degrees),hms_string(sun_ra.hours),
                      lat_dms_string(sun_dec.degrees),eqt_str,diff_str,sd_str]
        printout2.append(print_line)
    ttab=Table(printout2,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch],
               len(printout2)*[0.15*inch]) #create a table for the list
    total_days = days_in_month(imonth)
    if (total_days > 30): lastline = 30
    else: lastline = 25
    table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                  ('ALIGN',(1,0),(-1,-1),'CENTER'),
                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                  ('INNERGRID', (0,4), (-1,5), 0.25, colors.black),
                  ('INNERGRID', (0,9), (-1,10), 0.25, colors.black),
                  ('INNERGRID', (0,14), (-1,15), 0.25, colors.black),
                  ('INNERGRID', (0,19), (-1,20), 0.25, colors.black),
                  ('INNERGRID', (0,24), (-1,25), 0.25, colors.black),
                  ('INNERGRID', (0,lastline-1), (-1,lastline), 0.25, colors.black),
                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                  ('BOX', (1,0), (2,-1), 0.25, colors.black),
                  ('BOX', (2,0), (3,-1), 0.25, colors.black),
                  ('BOX', (3,0), (4,-1), 0.25, colors.black),
                  ('BOX', (4,0), (5,-1), 0.25, colors.black),
                  ('BOX', (5,0), (6,-1), 0.25, colors.black)]
    ttab.setStyle(TableStyle(table_args))
    elements.append(ttab)
    elements.append(PageBreak())

In [6]:
def get_ra_dec(body,iyear, imonth, dday, dhour):
    """
    Returns the right ascension, declination, and distance from the earth of the body at the time given
    """
    t = ts.utc(iyear, imonth, dday, dhour, 0, 0.0)
    # Greenwich mean time (UTC)
    eqt = -1 * equation_of_time(t) 
    t = ts.utc(iyear, imonth, dday, dhour, eqt,0 ) 
    # Greenwich apparent time (GAT)
    astrometric = earth.at(t).observe(body)
    apparent = earth.at(t).observe(body).apparent()
    ra, dec, distance = apparent.radec(epoch='date')
    return ra, dec, distance

In [7]:
def print_lunar_month(iyear,imonth):
    """
    Prints a table of lunar latitudes and longitudes for the year and month given
    Values at noon and midnight are given for each day in month
    """
    lstr = "The Moon's"
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    elements.append(Indenter(left=1.4*inch))
    header = Paragraph(ptext, ps)
    elements.append(header)
    elements.append(Indenter(left=-1.4*inch))
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[3.55*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    tabdata= [['','Longitude','Latitude']]
    ttab=Table(tabdata,[0.55*inch,1.5*inch,1.5*inch],[0.3*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black)]))
    elements.append(ttab)
    tabdata= [['day','date','Noon','Midnight','Noon','Midnight'],
             ['','','D.M.S','D.M.S','D.M.S','D.M.S']]
    ttab=Table(tabdata,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch],2*[0.2*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black)]))
    elements.append(ttab)
    printout2 = []
    #print(calendar.month_abbr[imonth]+" "+str(iyear)+"  "+"The Moon's")
    #print('day\t date\t Longitude\t \t Latitude')
    #print('\t \t Noon\t Midnight\t Noon\t Midnight')
    for iday in range(1, days_in_month(imonth)+1):
        dday = iday
        dhour = 12 #noon
        t = ts.utc(iyear, imonth, dday, dhour, 0, 0.0)
        # Greenwich mean time (UTC)
        eqt = -1 * equation_of_time(t) 
        t = ts.utc(iyear, imonth, dday, dhour, eqt,0 ) 
        moon_ra, moon_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)
        moon_horizontal_parallax = ((math.degrees(math.asin(6378.14/(distance.au*149597870)))*60))
        moon_semi_diameter = ((358473400/(distance.au * 149597870))/60);
        sd_str = str(int(moon_semi_diameter))+"' "+str(int((moon_semi_diameter - math.trunc(moon_semi_diameter))*60))+'"'
        eclat, eclon, ecd = earth.at(t).observe(moon).ecliptic_latlon()
        noon_lon = eclon
        noon_lat = eclat
        dhour = dhour + 12 #midnight
        t = ts.utc(iyear, imonth, dday, dhour, 0, 0.0)
        # Greenwich mean time (UTC)
        eqt = -1 * equation_of_time(t) 
        t = ts.utc(iyear, imonth, dday, dhour, eqt,0 ) 
        eclat, eclon, ecd = earth.at(t).observe(moon).ecliptic_latlon()
        dow = weekDay(iyear, imonth, iday)
        #print(dow,"\t",iday,"\t",dms_string(noon_lon.degrees),"\t",dms_string(eclon.degrees),"\t",
              #lat_dms_string(noon_lat.degrees),"\t",lat_dms_string(eclat.degrees))
        print_line = [str(dow),str(iday),dms_string2(noon_lon.degrees),dms_string2(eclon.degrees),
                      lat_dms_string(noon_lat.degrees),lat_dms_string(eclat.degrees)]
        printout2.append(print_line)
    ttab=Table(printout2,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.75*inch,0.75*inch],
               len(printout2)*[0.15*inch]) #create a table for the list
    total_days = days_in_month(imonth)
    if (total_days > 30): lastline = 30
    else: lastline = 25
    table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                  ('ALIGN',(1,0),(-1,-1),'CENTER'),
                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                  ('INNERGRID', (0,4), (-1,5), 0.25, colors.black),
                  ('INNERGRID', (0,9), (-1,10), 0.25, colors.black),
                  ('INNERGRID', (0,14), (-1,15), 0.25, colors.black),
                  ('INNERGRID', (0,19), (-1,20), 0.25, colors.black),
                  ('INNERGRID', (0,24), (-1,25), 0.25, colors.black),
                  ('INNERGRID', (0,lastline-1), (-1,lastline), 0.25, colors.black),
                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                  ('BOX', (1,0), (2,-1), 0.25, colors.black),
                  ('BOX', (2,0), (3,-1), 0.25, colors.black),
                  ('BOX', (3,0), (4,-1), 0.25, colors.black),
                  ('BOX', (4,0), (5,-1), 0.25, colors.black),
                  ('BOX', (5,0), (6,-1), 0.25, colors.black)]
    ttab.setStyle(TableStyle(table_args))
    elements.append(ttab)
    elements.append(PageBreak())

In [8]:
def print_lunar_month2(iyear,imonth,RAorDEC): # RA and Dec
    """
    Prints a table of the moon's age, meridian passage AND right ascension OR declination
    Each value given at 3 hr intervals starting at noon for each day in month
    RAorDEC = 0 is right ascension, != 0 is declination
    """
    lstr = "The Moon's"
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    elements.append(Indenter(left=0.0*inch))
    header = Paragraph(ptext, ps)
    elements.append(header)
    elements.append(Indenter(left=0.0*inch))
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[6.55*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    if (RAorDEC == 0):
        tabdata= [['','Meridian','Right Ascension']]
    else:
        tabdata= [['','Meridian','Declination']]
    ttab=Table(tabdata,[0.85*inch,0.5*inch,5.2*inch],[0.3*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(1,-1),'BOTTOM'),
                           ('VALIGN',(2,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (1,-1), 8), 
                           ('FONTSIZE', (2, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                             ]))
    elements.append(ttab)
    tabdata= [['day','date','Age','Passage','Noon','IIIh','VIh','IXh',"Midn/t",'XVh','XVIIIh','XXIh'],
             ['','','Days','H.M','D.M.S','D.M.S','D.M.S','D.M.S','D.M.S','D.M.S','D.M.S','D.M.S']]
    ttab=Table(tabdata,[0.3*inch,0.25*inch,0.3*inch,0.5*inch,0.65*inch,0.65*inch,0.65*inch,0.65*inch,
                       0.65*inch,0.65*inch,0.65*inch,0.65*inch],2*[0.2*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black),
                           ('BOX', (6,0), (7,-1), 0.25, colors.black),
                           ('BOX', (7,0), (8,-1), 0.25, colors.black),
                           ('BOX', (8,0), (9,-1), 0.25, colors.black),
                           ('BOX', (9,0), (10,-1), 0.25, colors.black)]))
    elements.append(ttab)
    printout2 = []
    #print(calendar.month_abbr[imonth]+" "+str(iyear)+"  "+"The Moon's")
    #print('day\t date\t Age\t Merid\t RA\t \t Dec')
    #print('\t \t \t Passage\t Noon\t Midnight\t Noon\t Midnight')
    for iday in range(1, days_in_month(imonth)+1):
        dday = iday
        
        dhour = 12 #noon
        sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        noon_ra, noon_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)
        if (iday == 1):
            moon_age = int((abs(noon_ra._degrees-sun_ra._degrees)/15/24)*29.5)
            if (moon_age > 29): moon_age = 1
        else:
            moon_age += 1
            if (moon_age > 29): moon_age = 1;
        #moon_horizontal_parallax = ((math.degrees(math.asin(6378.14/(distance.au*149597870)))*60))
        #moon_semi_diameter = ((358473400/(distance.au * 149597870))/60);
        #sd_str = str(int(moon_semi_diameter))+"' "+str(int((moon_semi_diameter - math.trunc(moon_semi_diameter))*60))+'"'

        dhour = 12+3 # 3PM
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        three_ra, three_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+6 # 6PM
       #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        six_ra, six_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+9 # 9PM
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        nine_ra, nine_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+15 # 3AM
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        fifteen_ra, fifteen_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+18 # 6AM
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        eighteen_ra, eighteen_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+21 # 9AM
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        twentyone_ra, twentyone_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)

        dhour = 12+12 #midnight
        #sun_ra, sun_dec, sun_distance = get_ra_dec(sun,iyear,imonth,dday,dhour)
        midnight_ra, midnight_dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)
        merid_pass = (sun_ra._degrees-noon_ra._degrees)/15
        if (merid_pass > 0): merid_pass = 24 - merid_pass
        else: merid_pass = abs(merid_pass)
        if (abs(midnight_ra._degrees-noon_ra._degrees) > 30): 
            merid_pass = merid_pass * (12/(12-((midnight_ra._degrees+360-noon_ra._degrees)/15)))
        else: 
            merid_pass = merid_pass * (12/(12-((midnight_ra._degrees-noon_ra._degrees)/15)));
        #merid_pass := merid_pass / sun_motion; 
        dow = weekDay(iyear, imonth, iday)
        #print(dow,"\t",iday,"\t",moon_age,"\t",hm_string(merid_pass),"\t",dms_string(noon_ra._degrees),"\t",dms_string(ra._degrees),"\t",
              #lat_dms_string(noon_dec.degrees),"\t",lat_dms_string(dec.degrees))
        if (RAorDEC == 0): # print RA
            print_line = [str(dow),str(iday),moon_age,hm_string(merid_pass),
                      dms_string2(noon_ra._degrees),dms_string2(three_ra._degrees),dms_string2(six_ra._degrees),
                      dms_string2(nine_ra._degrees),dms_string2(midnight_ra._degrees),dms_string2(fifteen_ra._degrees),
                      dms_string2(eighteen_ra._degrees),dms_string2(twentyone_ra._degrees)]
        else: # print Dec
            print_line = [str(dow),str(iday),moon_age,hm_string(merid_pass),
                          lat_dms_string(noon_dec.degrees),lat_dms_string(three_dec.degrees),
                          lat_dms_string(six_dec.degrees),lat_dms_string(nine_dec.degrees),
                          lat_dms_string(midnight_dec.degrees),lat_dms_string(fifteen_dec.degrees),
                          lat_dms_string(eighteen_dec.degrees),lat_dms_string(twentyone_dec.degrees)]
        printout2.append(print_line)
    ttab=Table(printout2,[0.3*inch,0.25*inch,0.3*inch,0.5*inch,0.65*inch,0.65*inch,0.65*inch,0.65*inch,
               0.65*inch,0.65*inch,0.65*inch,0.65*inch],len(printout2)*[0.15*inch]) #create a table for the list
    total_days = days_in_month(imonth)
    if (total_days > 30): lastline = 30
    else: lastline = 25
    table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                  ('ALIGN',(1,0),(-1,-1),'CENTER'),
                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                  ('INNERGRID', (0,4), (-1,5), 0.25, colors.black),
                  ('INNERGRID', (0,9), (-1,10), 0.25, colors.black),
                  ('INNERGRID', (0,14), (-1,15), 0.25, colors.black),
                  ('INNERGRID', (0,19), (-1,20), 0.25, colors.black),
                  ('INNERGRID', (0,24), (-1,25), 0.25, colors.black),
                  ('INNERGRID', (0,lastline-1), (-1,lastline), 0.25, colors.black),
                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                  ('BOX', (1,0), (2,-1), 0.25, colors.black),
                  ('BOX', (2,0), (3,-1), 0.25, colors.black),
                  ('BOX', (3,0), (4,-1), 0.25, colors.black),
                  ('BOX', (4,0), (5,-1), 0.25, colors.black),
                  ('BOX', (5,0), (6,-1), 0.25, colors.black),
                  ('BOX', (6,0), (7,-1), 0.25, colors.black),
                  ('BOX', (7,0), (8,-1), 0.25, colors.black),
                  ('BOX', (8,0), (9,-1), 0.25, colors.black),
                  ('BOX', (9,0), (10,-1), 0.25, colors.black)]
    ttab.setStyle(TableStyle(table_args))
    elements.append(ttab)
    elements.append(PageBreak())
                

In [9]:
def print_lunar_month3(iyear,imonth):
    """
    Prints a table of the moon's semi diameter, horizontal parallax and proportional log thereof
    Each value given for noon and midnight of each day
    """
    lstr = "The Moon's"
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    elements.append(Indenter(left=1.1*inch))
    header = Paragraph(ptext, ps)
    elements.append(header)
    elements.append(Indenter(left=-1.1*inch))
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[4.15*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    tabdata= [['','Semi Diameter','Horiz. Parallax','Proportional Log']]
    ttab=Table(tabdata,[0.55*inch,1.2*inch,1.2*inch,1.2*inch],[0.3*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (1,-1), 10), 
                           ('FONTSIZE', (2, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                             ]))
    elements.append(ttab)
    tabdata= [['day','date','Noon','Midnight','Noon','Midnight','Noon','Midnight'],
             ['','','M.S','M.S','M.S','M.S','','']]
    ttab=Table(tabdata,[0.3*inch,0.25*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch],2*[0.2*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black)]))
    elements.append(ttab)
    printout2 = []
    #print(calendar.month_abbr[imonth]+" "+str(iyear)+"  "+"The Moon's")
    #print('day\t date\t Semi Diameter\t H. Parallax\t \t Prop log\t')
    #print('\t \t Noon\t Midnight\t Noon\t Midnight\t Noon\t Midnight')
    for iday in range(1, days_in_month(imonth)+1):
        dday = iday
        dhour = 12 #noon
        ra, dec, noon_distance = get_ra_dec(moon,iyear,imonth,dday,dhour)
        noon_horizontal_parallax = ((math.degrees(math.asin(6378.14/(noon_distance.au*149597870)))*60))
        noon_hp_str = ms_string(noon_horizontal_parallax)
        noon_semi_diameter = ((358473400/(noon_distance.au * 149597870))/60);
        noon_sd_str = ms_string(noon_semi_diameter)
        noon_plog_str = str((math.log10(10800) - math.log10(math.trunc(noon_horizontal_parallax*60))))
        dhour = dhour + 12 #midnight
        ra, dec, distance = get_ra_dec(moon,iyear,imonth,dday,dhour)
        horizontal_parallax = ((math.degrees(math.asin(6378.14/(distance.au*149597870)))*60))
        hp_str = ms_string(horizontal_parallax)
        semi_diameter = ((358473400/(distance.au * 149597870))/60);
        sd_str = ms_string(semi_diameter)
        plog_str = str((math.log10(10800) - math.log10(math.trunc(horizontal_parallax*60))))
        dow = weekDay(iyear, imonth, iday)
        #print(dow,"\t",iday,"\t",noon_sd_str,"\t",sd_str,"\t",noon_hp_str,"\t",hp_str,"\t",noon_plog_str[2:6],"\t",plog_str[2:6])
        print_line = [str(dow),str(iday),noon_sd_str,sd_str,noon_hp_str,hp_str,noon_plog_str[2:6],plog_str[2:6]]
        printout2.append(print_line)
    ttab=Table(printout2,[0.3*inch,0.25*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch,0.6*inch],
               len(printout2)*[0.15*inch]) #create a table for the list
    total_days = days_in_month(imonth)
    if (total_days > 30): lastline = 30
    else: lastline = 25
    table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                  ('ALIGN',(1,0),(-1,-1),'CENTER'),
                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                  ('INNERGRID', (0,4), (-1,5), 0.25, colors.black),
                  ('INNERGRID', (0,9), (-1,10), 0.25, colors.black),
                  ('INNERGRID', (0,14), (-1,15), 0.25, colors.black),
                  ('INNERGRID', (0,19), (-1,20), 0.25, colors.black),
                  ('INNERGRID', (0,24), (-1,25), 0.25, colors.black),
                  ('INNERGRID', (0,lastline-1), (-1,lastline), 0.25, colors.black),
                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                  ('BOX', (1,0), (2,-1), 0.25, colors.black),
                  ('BOX', (2,0), (3,-1), 0.25, colors.black),
                  ('BOX', (3,0), (4,-1), 0.25, colors.black),
                  ('BOX', (4,0), (5,-1), 0.25, colors.black),
                  ('BOX', (5,0), (6,-1), 0.25, colors.black)]
    ttab.setStyle(TableStyle(table_args))
    elements.append(ttab)
    elements.append(PageBreak())

In [10]:
def print_planets_month(iyear,imonth):
    """
    Prints a table of the visible planet's right ascension, declination, and meridian passage
    Each value given for noon and midnight of each day
    """
    lstr = "The Planet's"
    ptext = "<font size=12>Lunar almanac for "+month_name(imonth)+" "+str(iyear)+"</font>"
    styleSheet = getSampleStyleSheet()
    styleSheet.fontName = 'Times-Roman'
    ps = ParagraphStyle(
            name='Total',
            fontSize=10,
            fontName='Times-Roman')
    elements.append(Indenter(left=1.9*inch))
    header = Paragraph(ptext, ps)
    elements.append(header)
    elements.append(Indenter(left=-1.9*inch))
    elements.append(Spacer(0, 0.2 * inch))
    tabdata= [[lstr]]
    ttab=Table(tabdata,[2.55*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 10), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ]))
    elements.append(ttab)
    tabdata= [['','Right Ascension','Declination','Meridian']]
    ttab=Table(tabdata,[0.55*inch,0.75*inch,0.75*inch,0.5*inch],[0.3*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(3,0),(-1,-1),'BOTTOM'),
                           ('VALIGN',(0,0),(2,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                             ]))
    elements.append(ttab)
    tabdata= [['day','date','H.M.S','D.M.S','Passage']]
    ttab=Table(tabdata,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.5*inch],[0.2*inch])
    ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                           ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                           ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                           ('FONTSIZE', (0, 0), (-1,-1), 8), 
                           ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                           ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (1,0), (2,-1), 0.25, colors.black),
                           ('BOX', (2,0), (3,-1), 0.25, colors.black),
                           ('BOX', (3,0), (4,-1), 0.25, colors.black),
                           ('BOX', (4,0), (5,-1), 0.25, colors.black),
                           ('BOX', (5,0), (6,-1), 0.25, colors.black)]))
    elements.append(ttab)
    printout2 = []
    weeks = [1,7,13,19,25]
    #print(calendar.month_abbr[imonth]+" "+str(iyear)+"  "+"The Planet's")
    #print('day\t date\t RA\t Dec\t Merid. Pass.')
    for k in range(10,14): 
        body = lunar_stars[k]
        #print(star_name[k])
        tabdata= [[star_name[k]]]
        ttab=Table(tabdata,[2.55*inch],[0.2*inch])
        ttab.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                                  ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                                  ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                                  ('FONTSIZE', (0, 0), (-1,-1), 8), 
                                  ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                                  ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                                  ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                                  ]))
        elements.append(ttab)
        for iday in weeks:
            dday = iday
            dhour = 12 #noon
            t = ts.utc(iyear, imonth, dday, dhour, 0, 0.0)
            # Greenwich mean time (UTC)
            eqt = -1 * equation_of_time(t) 
            t = ts.utc(iyear, imonth, dday, dhour, eqt,0 ) 
            # Greenwich apparent time (GAT)
            astrometric = earth.at(t).observe(sun)
            apparent = earth.at(t).observe(sun).apparent()
            sun_ra, sun_dec, sun_distance = apparent.radec(epoch='date')
            astrometric = earth.at(t).observe(body)
            apparent = earth.at(t).observe(body).apparent()
            ra, dec, distance = apparent.radec(epoch='date')
            noon_ra = ra
            noon_dec = dec
            merid_pass = (sun_ra._degrees-noon_ra._degrees)/15
            if (merid_pass > 0): merid_pass = 24 - merid_pass
            else: merid_pass = abs(merid_pass)
            if (abs(ra._degrees-noon_ra._degrees) > 30): 
                merid_pass = merid_pass * (12/(12-((ra._degrees+360-noon_ra._degrees)/15)))
            else: 
                merid_pass = merid_pass * (12/(12-((ra._degrees-noon_ra._degrees)/15)));
            dow = weekDay(iyear, imonth, iday)
            #print(dow,"\t",iday,"\t",hms_string(ra._degrees/15),"\t",lat_dms_string(dec.degrees),"\t",hm_string(merid_pass))
            print_line = [str(dow),str(iday),hms_string(ra._degrees/15),lat_dms_string(dec.degrees),hm_string(merid_pass)]
            printout2.append(print_line)
        ttab=Table(printout2,[0.3*inch,0.25*inch,0.75*inch,0.75*inch,0.5*inch],
                   len(printout2)*[0.15*inch]) #create a table for the list
        total_days = days_in_month(imonth)
        table_args = [('ALIGN',(0,0),(0,-1),'LEFT'),
                    ('ALIGN',(1,0),(-1,-1),'CENTER'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('TEXTCOLOR',(0,0),(-1,-1),colors.black),
                    ('FONTSIZE', (0, 0), (-1,-1), 8), 
                    ('FONTNAME', (0, 0), (-1,-1), 'Times-Roman'), 
                    ('INNERGRID', (0,4), (-1,5), 0.25, colors.black),
                    ('INNERGRID', (0,9), (-1,10), 0.25, colors.black),
                    ('INNERGRID', (0,14), (-1,15), 0.25, colors.black),
                    ('INNERGRID', (0,19), (-1,20), 0.25, colors.black),
                    ('INNERGRID', (0,24), (-1,25), 0.25, colors.black),
                    ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                    ('BOX', (1,0), (2,-1), 0.25, colors.black),
                    ('BOX', (2,0), (3,-1), 0.25, colors.black),
                    ('BOX', (3,0), (4,-1), 0.25, colors.black),
                    ('BOX', (4,0), (5,-1), 0.25, colors.black),
                    ('BOX', (5,0), (6,-1), 0.25, colors.black)]
        ttab.setStyle(TableStyle(table_args))
        elements.append(ttab)
        printout2 = []
    elements.append(PageBreak())

In [11]:
# MAIN PROGRAM * * *

# proportional logs
# plog(x) = log(10800) - log(x)   (there are 10800 sec in 3 hrs)
# plog(t-t1) = plog(d-D1) - plog(D2-D1) 
# t is time of interest, t1 is earlier time in almanac (e.g. IIIh)
# d is lunar distance at time t
# D1 is lunar dist at time t1, D2 is lunar distance 3 hours later

# angles between two bodies
# cos(A) = sin(Decl.1)sin(Decl.2) + cos(Decl.1)cos(Decl.2)cos(RA.1 - RA.2) 

imonth = 5    #    * * * * * * SET DATE HERE * * * * * * *
iyear = 2018

fname = month_name(imonth)+str(iyear)+".pdf"
doc = SimpleDocTemplate(fname, pagesize=letter)
# container for the 'Flowable' objects
elements = []
#planets = load('de422.bsp') #3000BCE to 3000AD
planets = load('de421.bsp') #small but only gives about 200 years

earth = planets["earth"]
sun = planets["sun"]
moon = planets["moon"]
mars = planets["mars"]
venus = planets["venus"]
jupiter = planets[5]
saturn = planets[6]
# Star positions are for epoch J2000
hamal = Star(ra_hours=(2, 7, 10.4), dec_degrees=(23, 27, 44.7),
            ra_mas_per_year=189,dec_mas_per_year=-148)
aldebaran = Star(ra_hours=(4, 35, 55.2), dec_degrees=(16, 30, 33.5),
            ra_mas_per_year=63,dec_mas_per_year=-190)
pollux = Star(ra_hours=(7, 45, 18.9), dec_degrees=(28, 1, 34.3),
            ra_mas_per_year=-627,dec_mas_per_year=-46)
regulus = Star(ra_hours=(10, 8, 22.3), dec_degrees=(11, 58, 1.9),
            ra_mas_per_year=-249,dec_mas_per_year=6)
spica = Star(ra_hours=(13, 25, 11.6), dec_degrees=(-11, 9, 40.7),
            ra_mas_per_year=-42,dec_mas_per_year=-31)
antares = Star(ra_hours=(16, 29, 24.4), dec_degrees=(-26, 25, 55.2),
            ra_mas_per_year=-12,dec_mas_per_year=-23)
altair = Star(ra_hours=(19, 50, 47), dec_degrees=(8, 52, 6),
            ra_mas_per_year=536,dec_mas_per_year=385)
fomalhaut = Star(ra_hours=(22, 57, 39), dec_degrees=(-29, 37, 20),
            ra_mas_per_year=329,dec_mas_per_year=-165)
markab = Star(ra_hours=(23, 4, 45.6), dec_degrees=(15, 12, 19),
            ra_mas_per_year=60,dec_mas_per_year=-41)

# Creates a list containing 31 lists, each of 8 lists, each of 14 items, all set to 0
# lunar_dist[day,j,body] where j is hour (noon,3,6,...,21)
d, jj, b = 32, 8, 14;

#bodies = [sun,hamal,aldebaran,pollux,regulus,spica,antares,altair,fomalhaut,markab,venus,mars,jupiter,saturn]
bodies = pd.Series([sun,hamal,aldebaran,pollux,regulus,spica,antares,altair,fomalhaut,markab,venus,mars,jupiter,saturn])

lunar_dist = [[[0 for x in range(d)] for y in range(jj)] for z in bodies] 
lunar_dir = [[[0 for x in range(d)] for y in range(jj)] for z in bodies] 
#lunar_dist = pd.DataFrame([[[0 for x in range(d)] for y in range(jj)] for z in bodies])
#lunar_dir = pd.DataFrame([[[0 for x in range(d)] for y in range(jj)] for z in bodies])
lunar_stars = [sun,hamal,aldebaran,pollux,regulus,spica,antares,altair,fomalhaut,markab,venus,mars,jupiter,saturn]

star_name = ['Sun','Hamal','Aldebaran','Pollux','Regulus','Spica','Antares','Altair','Fomalhaut','Markab','Venus','Mars','Jupiter','Saturn']

ts = load.timescale()
t = ts.now()

old_diff = 0
print('Lunar almanac for',month_name(imonth),iyear)

get_lunars(iyear,imonth) #stars

#print("timed get_lunars:",timeit.timeit(get_lunars(iyear,imonth)))
print('Printing lunar distance table')
printout = [[] for i in range(100)] #list of 100 lists
printout2 = [["","","Noon","IIIh","VIh","IXh","Midn/t","XVh","XVIIIh","XXIh"],
             ["Star's Name","Date","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S."]]
print_lunar_distances(2,True) #East of moon
printout = [[] for i in range(100)] #list of 100 lists
printout2 = [["","","Noon","IIIh","VIh","IXh","Midn/t","XVh","XVIIIh","XXIh"],
             ["Star's Name","Date","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S."]]
print_lunar_distances(1,True) #West of moon
printout = [[] for i in range(100)] #list of 100 lists
printout2 = [["","","Noon","IIIh","VIh","IXh","Midn/t","XVh","XVIIIh","XXIh"],
             ["Star's Name","Date","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S."]]
print_lunar_distances(2,False) #East of moon
printout = [[] for i in range(100)] #list of 100 lists
printout2 = [["","","Noon","IIIh","VIh","IXh","Midn/t","XVh","XVIIIh","XXIh"],
             ["Star's Name","Date","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S.","D.M.S."]]
print_lunar_distances(1,False) #West of moon
#print()
print('Printing Sun table')
print_solar_month(iyear,imonth) # Sun's long, RA, Dec
#print()
#print_lunar_month(iyear,imonth) # Moon's long and lat
#print()
print('Printing Moon tables 1','/ 3','\r', end="")
print_lunar_month2(iyear,imonth,0) # Moon's RA 
print('Printing Moon tables 2','/ 3','\r', end="")
print_lunar_month2(iyear,imonth,1) # Moon's Dec
print('Printing Moon tables 3','/ 3')
print_lunar_month3(iyear,imonth) # Moon's semi-diam, HP, plog
print('Printing Planets table')
print_planets_month(iyear,imonth) # Planets RA and Dec
# write the pdf document to disk
doc.build(elements, onFirstPage=addPageNumber, onLaterPages=addPageNumber)
print("done")

[#################################] 100% deltat.data
[#################################] 100% deltat.preds


Lunar almanac for May 2018
Calculating 31 / 31 
Printing lunar distance table
Printing Sun table
Printing Moon tables 3 / 3 
Printing Planets table
done
